# **Prueba tecnica Azzorti** 

## **Creacion del dataframe usando pandas**


**Proceso**

1. **Generación de datos:**
   - **Producto:** Se crea una lista con 1000 productos, numerados del 1 al 1000, con formato de 4 dígitos (ej. P0001, P0002,... P1000).
   - **Ventas:** Se generan 1000 valores aleatorios para las ventas, en el rango de 1000 a 10000, utilizando `np.random.randint()`. La semilla `np.random.seed(2)` asegura que los resultados sean reproducibles.
   - **Categoría:** Se asigna a cada producto una categoría basada en la paridad de su número de producto (es decir, el número después de la letra 'P'). Si el número es impar, la categoría será "GIMPAR", y si es par, será "GPAR".

2. **Creación del DataFrame:**
   - Se crea el DataFrame `df` con tres columnas: "Producto", "Ventas" y "Categoría".
   - La columna "Ventas" se asegura de que sea de tipo `float` mediante `df["Ventas"].astype(float)`.

3. **Revisión inicial:**
   - Se visualizan las primeras filas del DataFrame con `df.head()` para verificar que los datos fueron generados correctamente.




In [61]:
import pandas as pd
import numpy as np

# Generar la columna Producto con formato de 4 dígitos
producto = [f"P{i:04d}" for i in range(1, 1001)]


# Generar Ventas en el rango de 1000 a 10000
np.random.seed(2)#Permite que siempre me de los mismos datos
ventas = np.random.randint(1000, 10001, size=1000)

# Asignar Categoría según la paridad del número convertido a entero
#categoria = ["GIMPAR" if int(p[1:]) % 2 != 0 else "GPAR" for p in producto]
def categoria(producto):
    if int(producto[1:]) % 2 !=0:
        return "GIMPAR"
    else:
        return "GPAR"
categoria = [categoria(producto) for producto in producto]
# Crear el DataFrame 
df = pd.DataFrame({"Producto": producto, "Ventas": ventas, "Categoría": categoria})
df["Ventas"]=df["Ventas"].astype(float)
# Mostrar las primeras filas
df.head()



,Producto,Ventas,Categoría
0,P0001,8336.0,GIMPAR
1,P0002,3575.0,GPAR
2,P0003,7637.0,GIMPAR
3,P0004,3514.0,GPAR
4,P0005,2099.0,GIMPAR


## **Creación de columna de venta neta**

1. **Generación de la variable `Venta Neta`**
   Una vez generado el DataFrame, se crea una nueva columna llamada `Venta Neta` que se calcula de la siguiente manera:
   - Se aplica un impuesto general a las ventas del 18% a cada valor de `Ventas`.
   - Se aplica un descuento adicional basado en escalas de venta:

     | Rango de Venta   | Descuento  |
     |------------------|------------|
     | 1000 - 1500      | 1%         |
     | 1500 - 3000      | 2%         |
     | 3000 - 7000      | 3%         |
     | 7000 - 9000      | 4%         |
     | 10000 >          | 5%         |

2. **Función de descuentos:**
   Se define la función `descuentos(row)` que aplica tanto el impuesto como el descuento correspondiente según el valor de la venta
3. **Aplicación de la función:**
    Se utiliza el método `apply()` de pandas para aplicar la función descuentos a cada valor de la columna `Ventas` y crear la nueva columna `Venta Neta`.


   


In [62]:
def descuentos(row):
    if row >= 1000 and row < 1500:
        return int(round(row - (row * 0.19), 0))
    if row >= 1500 and row < 3000:
        return int(round(row - (row * 0.20), 0))
    if row >= 3000 and row < 7000:
        return int(round(row - (row * 0.21), 0))
    if row >= 7000 and row < 9000:  
        return int(round(row - (row * 0.22), 0))
    if row >= 9000 and row <= 10000:
        return int(round(row - (row * 0.18), 0))
    if row > 10000:
        return int(round(row - (row * 0.23), 0))

# Aplicando la función a la columna "Ventas"
df["Venta Neta"] = df["Ventas"].apply(lambda x: descuentos(x))

# Mostrar las primeras filas del dataframe
df.head()


,Producto,Ventas,Categoría,Venta Neta
0,P0001,8336.0,GIMPAR,6502
1,P0002,3575.0,GPAR,2824
2,P0003,7637.0,GIMPAR,5957
3,P0004,3514.0,GPAR,2776
4,P0005,2099.0,GIMPAR,1679


## **Exportacion del dataframe en fomato `parquet` y compresion `snappy`**

In [63]:
df.to_parquet("ventas.parquet", engine="pyarrow", compression="snappy")

## **Dashboard en PowerBi**

In [64]:
from IPython.display import HTML

# Código HTML del Dashboard de Power BI
html_code_powerbi_dashboard = """<iframe title="Dashboard" width="1300" height="700" src="https://app.powerbi.com/view?r=eyJrIjoiYTYzMGE1YmEtNjE5Yi00NGVlLWFhNjctMDA4OTdhOTUwMjM1IiwidCI6IjUwNjQwNTg0LTJhNDAtNDIxNi1hODRiLTliM2VlMGYzZjZjZiIsImMiOjR9" frameborder="0" allowFullScreen="true"></iframe>
"""

# Mostrar el Dashboard dentro del notebook
display(HTML(html_code_powerbi_dashboard))


## Adicional: **Conexión y Ejecución de Consulta SQL en PostgreSQL**
Además de trabajar con un DataFrame en Python, se creó una base de datos en PostgreSQL y se replicó la estructura del DataFrame como una tabla llamada ventas_productos. Posteriormente, se estableció una conexión con la base de datos, se ejecutó una consulta SQL y se extrajo la información de la tabla como un DataFrame en Python.
 1. **Conexión a la base de datos:**
   - Se establece una conexión con la base de datos PostgreSQL utilizando el módulo `psycopg2` y las credenciales proporcionadas (usuario, contraseña, nombre de la base de datos y host).

2. **Ejecución de la consulta:**
   - Se crea una función llamada `run_query(query)` que ejecuta una consulta SQL sobre la base de datos.
   - Dentro de la función:
     - Se establece la conexión con la base de datos.
     - Se crea un cursor para interactuar con la base de datos.
     - Se ejecuta la consulta SQL proporcionada mediante `cursor.execute(query)`.
     - Se recuperan todos los resultados de la consulta con `cursor.fetchall()`.
     - Se obtienen los nombres de las columnas de los resultados.
     - Se cierra tanto el cursor como la conexión a la base de datos.

3. **Conversión de resultados a DataFrame:**
   - Los resultados obtenidos se convierten en un DataFrame de pandas para facilitar su manejo y análisis.
   - Se crea un DataFrame con las filas obtenidas de la consulta y los nombres de las columnas.

4. **Visualización de los resultados:**
   - Se visualizan las primeras filas del DataFrame con `df.head()` para verificar que los datos fueron recuperados correctamente de la base de datos.


In [68]:
import psycopg2  # Asegúrate de que el módulo psycopg2 esté importado para usar sus funcionalidades
import pandas as pd  # Importa pandas para trabajar con DataFrames

def connect_to_database():
    # Esta función establece una conexión con la base de datos PostgreSQL usando las credenciales proporcionadas
    return psycopg2.connect(
        host="localhost",  # Nombre del host del servidor PostgreSQL
        database="pruebatecnicasql",  # Nombre de la base de datos a la que conectar
        user="pruebatecnicasql",  # Nombre de usuario para la autenticación
        password="pruebatecnicasql"  # Contraseña para la autenticación
    )

def run_query(query):
    # Esta función ejecuta una consulta SQL dada en la base de datos conectada y devuelve el resultado como un DataFrame de pandas
    
    db = connect_to_database()  # Establece la conexión con la base de datos
    cursor = db.cursor()  # Crea un objeto cursor para interactuar con la base de datos
    
    cursor.execute(query)  # Ejecuta la consulta SQL proporcionada
    result = cursor.fetchall()  # Obtiene todos los resultados de la consulta
    # Obtener nombres de las columnas
    #print(cursor.description)
    column_names = [desc[0] for desc in cursor.description]

    cursor.close()  # Cierra el cursor para liberar los recursos
    db.close()  # Cierra la conexión con la base de datos
    
    return pd.DataFrame(result, columns=column_names)  # Convierte el resultado a un DataFrame de pandas y lo devuelve

# Define tu consulta SQL
query = "SELECT * FROM ventas_productos"

# Ejecuta la consulta y almacena el resultado en un DataFrame
df = run_query(query)
df["Venta neta"]=df["ventas"].apply(lambda x:descuentos(x))
# Muestra las primeras filas del DataFrame
df.head()


,producto,ventas,categoría,Venta neta
0,P0001,5405.0,GIMPAR,4270
1,P0002,3564.0,GPAR,2816
2,P0003,5952.0,GIMPAR,4702
3,P0004,7325.0,GPAR,5714
4,P0005,9495.0,GIMPAR,7786


**QUERY DE PRUEBA**

In [66]:
query = "SELECT *, DENSE_RANK() OVER (ORDER BY ventas DESC) AS Rank FROM ventas_productos"
dfprueba1=run_query(query)
dfprueba1.head()


,producto,ventas,categoría,rank
0,P0790,9985.0,GPAR,1
1,P0705,9977.0,GIMPAR,2
2,P0190,9963.0,GPAR,3
3,P0067,9953.0,GIMPAR,4
4,P0249,9941.0,GIMPAR,5


usando pandas

In [72]:
df.sort_values(by='ventas',ascending=False)[:5]

,producto,ventas,categoría,Venta neta
789,P0790,9985.0,GPAR,8188
704,P0705,9977.0,GIMPAR,8181
189,P0190,9963.0,GPAR,8170
66,P0067,9953.0,GIMPAR,8161
248,P0249,9941.0,GIMPAR,8152


In [73]:
dfs=pd.read_excel('D:\Documentos\Github\Prueba_tecnica_Azzorti\BASES_MODELO.xlsx',sheet_name=None)

In [79]:
dfs.keys()


dict_keys(['DESCRIPICION', 'BASE_VARIABLES', 'BASE_ASESORA', 'BASE_ESTIMADOS'])

In [139]:
df_bases_variables=dfs['BASE_VARIABLES']
df_bases_variables.columns

Index(['CAMPANA', 'CODIGO_DEL_PRODUCTO', 'VENTA', 'VENTA_ZONA_101',
       'VENTA_ZONA_102', 'VENTA_ZONA_103', 'VENTA_ZONA_104', 'VENTA_ZONA_107',
       'VENTA_ZONA_109', 'VENTA_ZONA_110', 'VENTA_ZONA_111', 'VENTA_ZONA_112',
       'VENTA_ZONA_115', 'VENTA_ZONA_116', 'VENTA_ZONA_119', 'NOMB_SUBGRUPO',
       'TALLA', 'PRECIO_NAC'],
      dtype='object')

In [138]:
df_bases_estimados=dfs['BASE_ESTIMADOS']
df_bases_estimados.columns


Index(['CAMPANA', 'CODIGO_DEL_PRODUCTO', 'CANTIDADES_ESTIMADAS'], dtype='object')

Index(['CAMPANA', 'N° ASESORAS_ZONA_101', 'N° ASESORAS_ZONA_102',
       'N° ASESORAS_ZONA_103', 'N° ASESORAS_ZONA_104', 'N° ASESORAS_ZONA_107',
       'N° ASESORAS_ZONA_109', 'N° ASESORAS_ZONA_110', 'N° ASESORAS_ZONA_111',
       'N° ASESORAS_ZONA_112', 'N° ASESORAS_ZONA_115', 'N° ASESORAS_ZONA_116',
       'N° ASESORAS_ZONA_119', 'N° ASESORAS'],
      dtype='object')

In [143]:
common=[]
for i in df_bases_variables.columns:
        if i in df_bases_estimados.columns:
            common.append(i)
df_merge=df_bases_variables.merge(df_bases_estimados,how="inner",on=common)
df_merge

,CAMPANA,CODIGO_DEL_PRODUCTO,VENTA,VENTA_ZONA_101,VENTA_ZONA_102,VENTA_ZONA_103,VENTA_ZONA_104,VENTA_ZONA_107,VENTA_ZONA_109,VENTA_ZONA_110,VENTA_ZONA_111,VENTA_ZONA_112,VENTA_ZONA_115,VENTA_ZONA_116,VENTA_ZONA_119,NOMB_SUBGRUPO,TALLA,PRECIO_NAC,CANTIDADES_ESTIMADAS
0,201902,P00845,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900,46
1,201908,P03495,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-L,99900,14
2,201910,P04897,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-L,99900,33
3,201907,P03019,25,0,0,0,0,0,0,0,0,0,0,0,0,215 RE-FALDAS FEM,T-XS,99900,24
4,201903,P01075,25,0,0,0,0,0,0,0,0,0,0,0,0,216 RE-JEANS FEM,T-8,99900,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,201902,P00471,28597,162,216,0,0,0,0,0,162,162,108,324,216,216 RE-JEANS FEM,T-16,99900,67776
5813,201902,P00805,30571,1140,1026,589,266,1083,456,361,285,418,456,418,323,225 RE-VESTIDOS FEM,T-XL,99900,29960
5814,201901,P00127,31242,1842,1588,3112,508,1207,381,699,254,826,699,953,254,225 RE-VESTIDOS FEM,T-XL,99900,35303
5815,201902,P00562,33212,969,760,760,361,418,912,304,456,646,836,646,532,209 RE-CAPRIS FEM,T-XL,99900,61442


In [147]:
def union(df1,df2,columnaunion=""):
    common=[]
    for i in df1.columns:
        if i in df2.columns:
            common.append(i)
    if columnaunion:
        df_merge=df1.merge(df2,how="inner",on=columnaunion)   
    else:
        df_merge=df1.merge(df2,how="inner",on=common)
    return df_merge
dfunion=union(df_bases_variables,df_bases_estimados)
dfunion=union(dfunion,df_bases_asesoras)
dfunion.columns

Index(['CAMPANA', 'CODIGO_DEL_PRODUCTO', 'VENTA', 'VENTA_ZONA_101',
       'VENTA_ZONA_102', 'VENTA_ZONA_103', 'VENTA_ZONA_104', 'VENTA_ZONA_107',
       'VENTA_ZONA_109', 'VENTA_ZONA_110', 'VENTA_ZONA_111', 'VENTA_ZONA_112',
       'VENTA_ZONA_115', 'VENTA_ZONA_116', 'VENTA_ZONA_119', 'NOMB_SUBGRUPO',
       'TALLA', 'PRECIO_NAC', 'CANTIDADES_ESTIMADAS', 'N° ASESORAS_ZONA_101',
       'N° ASESORAS_ZONA_102', 'N° ASESORAS_ZONA_103', 'N° ASESORAS_ZONA_104',
       'N° ASESORAS_ZONA_107', 'N° ASESORAS_ZONA_109', 'N° ASESORAS_ZONA_110',
       'N° ASESORAS_ZONA_111', 'N° ASESORAS_ZONA_112', 'N° ASESORAS_ZONA_115',
       'N° ASESORAS_ZONA_116', 'N° ASESORAS_ZONA_119', 'N° ASESORAS'],
      dtype='object')

In [ ]:
common=[]
for i in df_bases_variables.columns:
    if i in df_bases_estimados.columns:
        common.append(i)


In [133]:
df_merge

,CAMPANA_x,CODIGO_DEL_PRODUCTO,VENTA,VENTA_ZONA_101,VENTA_ZONA_102,VENTA_ZONA_103,VENTA_ZONA_104,VENTA_ZONA_107,VENTA_ZONA_109,VENTA_ZONA_110,VENTA_ZONA_111,VENTA_ZONA_112,VENTA_ZONA_115,VENTA_ZONA_116,VENTA_ZONA_119,NOMB_SUBGRUPO,TALLA,PRECIO_NAC,CAMPANA_y,CANTIDADES_ESTIMADAS
0,201902,P00845,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900,201902,46
1,201908,P03495,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-L,99900,201908,14
2,201910,P04897,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-L,99900,201910,33
3,201907,P03019,25,0,0,0,0,0,0,0,0,0,0,0,0,215 RE-FALDAS FEM,T-XS,99900,201907,24
4,201903,P01075,25,0,0,0,0,0,0,0,0,0,0,0,0,216 RE-JEANS FEM,T-8,99900,201903,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,201902,P00471,28597,162,216,0,0,0,0,0,162,162,108,324,216,216 RE-JEANS FEM,T-16,99900,201902,67776
5813,201902,P00805,30571,1140,1026,589,266,1083,456,361,285,418,456,418,323,225 RE-VESTIDOS FEM,T-XL,99900,201902,29960
5814,201901,P00127,31242,1842,1588,3112,508,1207,381,699,254,826,699,953,254,225 RE-VESTIDOS FEM,T-XL,99900,201901,35303
5815,201902,P00562,33212,969,760,760,361,418,912,304,456,646,836,646,532,209 RE-CAPRIS FEM,T-XL,99900,201902,61442
